In [1]:
'''更新日志10July
把最初的回归模型改为分类模型，输入从百分制换成等级制：
损失函数从均方差改成交叉熵；
输出从1通道改为5通道。
'''

'更新日志10July\n把最初的回归模型改为分类模型，输入从百分制换成等级制：\n损失函数从均方差改成交叉熵；\n输出从1通道改为5通道。\n'

In [2]:
'''
双通道3D卷积神经网络，使用体素模型。相比点云方法，优点是能够直接进行3D卷积运算，缺点是数据量和算量都更大。
通道1：原始牙齿的体素模型
通道2：预备体的体素模型
网络结构：3D卷积-池化-3D卷积-池化-扁平化-MLP-输出
'''

'\n双通道3D卷积神经网络，使用体素模型。相比点云方法，优点是能够直接进行3D卷积运算，缺点是数据量和算量都更大。\n通道1：原始牙齿的体素模型\n通道2：预备体的体素模型\n网络结构：3D卷积-池化-3D卷积-池化-扁平化-MLP-输出\n'

In [3]:
'''
# 安装环境
pip install trimesh
'''

'\n# 安装环境\npip install trimesh\n'

In [4]:
'''
# 安装环境
pip install torch torchvision torchaudio
'''

'\n# 安装环境\npip install torch torchvision torchaudio\n'

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import trimesh
from sklearn.model_selection import KFold

In [6]:
# 加载和归一化模型
def load_mesh(file_path):
    mesh = trimesh.load(file_path)
    # 如果加载的是Scene对象，转换为Trimesh对象
    if isinstance(mesh, trimesh.Scene):
        mesh = mesh.dump(concatenate=True)
    return mesh

def normalize_mesh(mesh):
    # 将网格中心平移到原点
    mesh.apply_translation(-mesh.centroid)
    # 计算缩放比例
    scale_factor = 1.0 / max(mesh.extents)
    # 缩放网格
    mesh.apply_scale(scale_factor)
    return mesh

def mesh_to_voxel(mesh, voxel_size=32):
    # 将网格体素化
    voxelized_mesh = mesh.voxelized(pitch=1.0 / voxel_size)
    # 确保体素矩阵的形状为 (voxel_size, voxel_size, voxel_size)
    voxel_matrix = voxelized_mesh.matrix
    padded_matrix = np.zeros((voxel_size, voxel_size, voxel_size), dtype=voxel_matrix.dtype)
    shape = np.minimum(voxel_matrix.shape, (voxel_size, voxel_size, voxel_size))
    padded_matrix[:shape[0], :shape[1], :shape[2]] = voxel_matrix[:shape[0], :shape[1], :shape[2]]
    return padded_matrix

# 加载并归一化打磨前的初始模型
initial_mesh_path = 'incisor.obj'
initial_mesh = normalize_mesh(load_mesh(initial_mesh_path))
voxel_size = 32  # 体素网格尺寸
voxel_data_initial = mesh_to_voxel(initial_mesh, voxel_size)

class TeethDataset(Dataset):
    def __init__(self, initial_voxel, score_file_path, voxel_size=32):
        self.initial_voxel = initial_voxel
        self.indices, self.scores = read_scores(score_file_path)
        self.voxel_size = voxel_size

    def __len__(self):
        return len(self.scores)

    def __getitem__(self, idx):
        index = self.indices[idx]
        score = self.scores[idx]
        # 加载并归一化打磨后的模型
        mesh_path = f'{index}.obj'
        mesh = normalize_mesh(load_mesh(mesh_path))
        voxel_data_after = mesh_to_voxel(mesh, self.voxel_size)
        # 双通道数据
        sample = np.stack([self.initial_voxel, voxel_data_after], axis=0)
        return torch.tensor(sample, dtype=torch.float32), torch.tensor(score, dtype=torch.long)  # 确保 score 是 long 类型


In [7]:
# 读取评分
def read_scores(score_file_path):
    indices = []
    scores = []
    with open(score_file_path, 'r') as f:
        for line in f:
            index, score = line.strip().split()
            indices.append(int(index))
            scores.append(int(score)-1)  # 离散化，-1，取值区间[0,4]
    return indices, scores  

# 创建数据集
score_file_path = 'grades.txt'
dataset = TeethDataset(voxel_data_initial, score_file_path, voxel_size)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [8]:
# 定义模型
class Teeth3DCNN(nn.Module):
    def __init__(self):
        super(Teeth3DCNN, self).__init__()
        self.conv1 = nn.Conv3d(2, 32, kernel_size=3, stride=1, padding=1)  # 双通道输入
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8 * 8, 256)  # 体素尺寸为32*32*32，经过两次pooling变为8*8*8
        self.fc2 = nn.Linear(256, 5)  # 输出5个类别
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x    

# 创建模型
model = Teeth3DCNN()
criterion = nn.CrossEntropyLoss()  # 损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 学习率和优化器


In [9]:
# 训练模型

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

# 模型评估
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
print(f"Test Loss: {test_loss / len(dataloader)}")


Epoch 1/30, Loss: 1.5200162498574508
Epoch 2/30, Loss: 1.510844233788942
Epoch 3/30, Loss: 1.4929131865501404
Epoch 4/30, Loss: 1.4723143201125295
Epoch 5/30, Loss: 1.488644490116521
Epoch 6/30, Loss: 1.4415318903170133
Epoch 7/30, Loss: 1.4134152719849034
Epoch 8/30, Loss: 1.3580476798509296
Epoch 9/30, Loss: 1.3123042473667546
Epoch 10/30, Loss: 1.2349044711966264
Epoch 11/30, Loss: 1.2355233713200218
Epoch 12/30, Loss: 1.102948591897362
Epoch 13/30, Loss: 1.0829122097868669
Epoch 14/30, Loss: 1.0323126512138467
Epoch 15/30, Loss: 0.8911443580138055
Epoch 16/30, Loss: 0.7919988114582864
Epoch 17/30, Loss: 0.7203086461675795
Epoch 18/30, Loss: 0.6656947641780502
Epoch 19/30, Loss: 0.5962691628619244
Epoch 20/30, Loss: 0.43406012546467154
Epoch 21/30, Loss: 0.39069379439675495
Epoch 22/30, Loss: 0.4144342233868022
Epoch 23/30, Loss: 0.3455668440774867
Epoch 24/30, Loss: 0.2728414710022901
Epoch 25/30, Loss: 0.44919813762565974
Epoch 26/30, Loss: 0.18954822858550438
Epoch 27/30, Loss: 0

In [11]:
# 对150个预备体进行评分
model.eval()
scores = []
with torch.no_grad():
    for i in range(len(dataset)):
        inputs, _ = dataset[i]  # 忽略目标评分，只取输入数据
        inputs = inputs.unsqueeze(0)  # 添加批次维度
        output = model(inputs)
        predicted_class = torch.argmax(output, dim=1).item() + 1
        scores.append(predicted_class)

# 输出150个预备体的评分
for i, score in enumerate(scores):
    print(f'Tooth {i+1}: Score = {score}')


Tooth 1: Score = 4
Tooth 2: Score = 4
Tooth 3: Score = 3
Tooth 4: Score = 4
Tooth 5: Score = 4
Tooth 6: Score = 3
Tooth 7: Score = 4
Tooth 8: Score = 4
Tooth 9: Score = 4
Tooth 10: Score = 4
Tooth 11: Score = 1
Tooth 12: Score = 3
Tooth 13: Score = 3
Tooth 14: Score = 3
Tooth 15: Score = 3
Tooth 16: Score = 1
Tooth 17: Score = 3
Tooth 18: Score = 4
Tooth 19: Score = 3
Tooth 20: Score = 3
Tooth 21: Score = 4
Tooth 22: Score = 3
Tooth 23: Score = 4
Tooth 24: Score = 4
Tooth 25: Score = 5
Tooth 26: Score = 3
Tooth 27: Score = 4
Tooth 28: Score = 3
Tooth 29: Score = 3
Tooth 30: Score = 5
Tooth 31: Score = 3
Tooth 32: Score = 3
Tooth 33: Score = 4
Tooth 34: Score = 4
Tooth 35: Score = 3
Tooth 36: Score = 5
Tooth 37: Score = 2
Tooth 38: Score = 5
Tooth 39: Score = 3
Tooth 40: Score = 4
Tooth 41: Score = 4
Tooth 42: Score = 3
Tooth 43: Score = 5
Tooth 44: Score = 4
Tooth 45: Score = 3
Tooth 46: Score = 4
Tooth 47: Score = 4
Tooth 48: Score = 3
Tooth 49: Score = 4
Tooth 50: Score = 5
Tooth 51:

In [12]:
_, Real_Scores = read_scores(score_file_path)

for i, score in enumerate(scores):
    print(f'Tooth {i+1}: Score = {score}', f'Err = {score-Real_Scores[i]-1}' )

Tooth 1: Score = 4 Err = 0
Tooth 2: Score = 4 Err = 0
Tooth 3: Score = 3 Err = 0
Tooth 4: Score = 4 Err = 0
Tooth 5: Score = 4 Err = 0
Tooth 6: Score = 3 Err = 0
Tooth 7: Score = 4 Err = 0
Tooth 8: Score = 4 Err = 0
Tooth 9: Score = 4 Err = 0
Tooth 10: Score = 4 Err = 0
Tooth 11: Score = 1 Err = 0
Tooth 12: Score = 3 Err = 0
Tooth 13: Score = 3 Err = 0
Tooth 14: Score = 3 Err = 0
Tooth 15: Score = 3 Err = 0
Tooth 16: Score = 1 Err = 0
Tooth 17: Score = 3 Err = 0
Tooth 18: Score = 4 Err = 0
Tooth 19: Score = 3 Err = 0
Tooth 20: Score = 3 Err = 0
Tooth 21: Score = 4 Err = 0
Tooth 22: Score = 3 Err = 0
Tooth 23: Score = 4 Err = 0
Tooth 24: Score = 4 Err = 0
Tooth 25: Score = 5 Err = 0
Tooth 26: Score = 3 Err = 0
Tooth 27: Score = 4 Err = 0
Tooth 28: Score = 3 Err = 0
Tooth 29: Score = 3 Err = 0
Tooth 30: Score = 5 Err = 0
Tooth 31: Score = 3 Err = 0
Tooth 32: Score = 3 Err = 0
Tooth 33: Score = 4 Err = 0
Tooth 34: Score = 4 Err = 0
Tooth 35: Score = 3 Err = 0
Tooth 36: Score = 5 Err = 0
T